### Days: wps_climdex_days

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf) days functions

- [climdex.su](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=25) Computes the SU (summer days) climdexindex: the annual count of days where daily maximum temperature exceeds 25 degreesCelsius.
- [climdex.id](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=11) Computes the ID (icing days) climdexindex: the annual count of days where daily maximum temperature was below 0 degrees Celsius.
- [climdex.fd](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=8) Computes the FD frost days) climdexindex: the annual count of days where daily minimum temperature was below 0 degrees Celsius.
- [climdex.tr](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=32) Computees the TR (tropical nights) climdexindex: the annual count of days where daily minimum temperature stays above 20 degrees Celsius.

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from importlib.resources import files
from tempfile import NamedTemporaryFile

from wps_tools.output_handling import auto_construct_outputs
from wps_tools.R import get_robjects, construct_r_out, rda_to_vector, test_rda_output
from wps_tools.testing import get_target_url

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_days?

Signature:
quail.climdex_days(
    climdex_input,
    days_type=None,
    loglevel='INFO',
    output_file='output.rda',
    output_formats=None,
)
Docstring:
Takes a climdexInput object as input and computes the annual count of days where daily temperature satisfies some condition.
                "summer": the annual count of days where daily maximum temperature
                exceeds 25 degreesCelsius
                "icing": the annual count of days where daily maximum temperature
                was below 0 degrees Celsius
                "frost": the annual count of days where daily minimum temperature
                was below 0 degrees Celsius
                "tropical nights": the annual count of days where daily minimum
                temperature stays above 20 degrees Celsius

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    RDS or Rdata (.rds, .rda, .rdata) file containing R Object of type climdexInput
output_file : string
    Filename t

#### Run wps_climdex_days process for climdex.su (summer days) with rda input

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="summer_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=(files("tests") / "data/climdexInput.rda").resolve(),
            days_type="su",
            output_file=output_file.name,
        )
su_url = output.get()[0]

#### Run wps_climdex_days process for climdex.id (icing days) with rda input

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="icing_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=(files("tests") / "data/climdexInput.rda").resolve(),      
            days_type="id",
            output_file=output_file.name,
        )
id_url = output.get()[0]

#### Run wps_climdex_days process for climdex.fd (frost days) with rds input

In [8]:
with NamedTemporaryFile(suffix=".rda", prefix="frost_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=(files("tests") / "data/climdexInput.rda").resolve(),        
            days_type="fd",
            output_file=output_file.name,
        )
fd_url = output.get()[0]

#### Run wps_climdex_days process for climdex.tr (tropical nights) with rds input

In [9]:
with NamedTemporaryFile(suffix=".rda", prefix="frost_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=(files("tests") / "data/climdexInput.rds").resolve(),        
            days_type="tr",
            output_file=output_file.name,
        )
tr_url = output.get()[0]

#### Run with multiple inputs

In [10]:
climdex_inputs = [
    (files("tests") / "data/climdexInput.rds").resolve(),
    (files("tests") / "data/climdexInput.rda").resolve(),
    (files("tests") / "data/climdex_input_multiple.rda").resolve(),
]
with NamedTemporaryFile(suffix=".rda", prefix="frost_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=climdex_inputs,        
            days_type="su",
            output_file=output_file.name,
        )

In [11]:
# NBVAL_IGNORE_OUTPUT
construct_r_out(output.get())

[[R object with classes: ('numeric',) mapped to:
  [     nan, 11.000000,      nan,      nan, ..., 4.000000,      nan, 13.000000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 11.000000,      nan,      nan, ..., 4.000000,      nan, 13.000000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 11.000000,      nan,      nan, ..., 4.000000,      nan, 13.000000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 11.000000,      nan,      nan, ..., 4.000000,      nan, 13.000000,      nan]]]

Access the output with **rda_to_vector** or **construct_r_out** from **wps_tools.R**

In [13]:
# use print() to see whole vector
su = rda_to_vector(su_url, "su1_ci")
print(f"SUMMER DAYS\n{su}")
id_ = rda_to_vector(id_url, "id1_ci")
fd = rda_to_vector(fd_url, "fd1_ci")
tr = rda_to_vector(tr_url, "tr1_ci")

SUMMER DAYS
1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 
  NA   11   NA   NA    2   NA    3    4    6   11    6    4   10    5    6    4 
1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 
   1   NA   NA    5    3    2    5    3    4   NA   NA    6   NA    4    2    8 
1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 
  NA   10   NA   NA    7   12    8   15    2    8    4   NA   13   NA 



In [14]:
construct_r_out([su_url, id_url, fd_url, tr_url])

[[R object with classes: ('numeric',) mapped to:
  [     nan, 11.000000,      nan,      nan, ..., 4.000000,      nan, 13.000000,      nan]],
 [R object with classes: ('numeric',) mapped to:
  [     nan, 0.000000,      nan,      nan, ..., 0.000000,      nan, 0.000000,      nan]],
 [R object with classes: ('numeric',) mapped to:
  [     nan, 10.000000,      nan,      nan, ..., 1.000000,      nan, 0.000000,      nan]],
 [R object with classes: ('numeric',) mapped to:
  [     nan, 0.000000,      nan,      nan, ..., 0.000000,      nan, 0.000000,      nan]]]

#### Test output against expected output

In [15]:
test_rda_output(
        su_url, "su1_ci", "expected_days_data.rda", "expected_summer_days"
    )
test_rda_output(
        id_url, "id1_ci", "expected_days_data.rda", "expected_icing_days"
    )
test_rda_output(
        fd_url, "fd1_ci", "expected_days_data.rda", "expected_frost_days"
    )
test_rda_output(
        tr_url, "tr1_ci", "expected_days_data.rda", "expected_tropical_nights"
    )